In [ ]:
from PIL import Image, ImageDraw, ImageFont
from numpy.random.mtrand import randint
import matplotlib.pyplot as plt

def example_prediction(model, test_dataset, loss_yolo_fn = None, prob_threshold = 0.5, iou_threshold = 0.51):
  """
  Performs object detection on a randomly selected image from the test dataset using the provided model.

  Args:
      model (torch.nn.Module): The object detection model to be used for inference.
      test_dataset (torch.utils.data.Dataset): The dataset containing test images and their corresponding labels.
      loss_yolo_fn (torch.nn.Module, optional): The YOLO loss function used for calculating the test loss.
          Defaults to None.
      prob_threshold (float, optional): The probability threshold for filtering out low-confidence predictions.
          Defaults to 0.5.
      iou_threshold (float, optional): The IoU threshold for non-maximum suppression to remove redundant predictions.
          Defaults to 0.51.

  Returns:
      PIL.Image.Image: The annotated image with bounding boxes drawn around detected objects.
      float: The test loss if a YOLO loss function is provided; otherwise, 0.
  """
  # Randomly select an image and its corresponding labels from the test dataset
  n = randint(0,len(test_dataset) - 1)
  img, labels = test_dataset[n][0], test_dataset[n][1]
  img, labels = img.to(device), labels.to(device) # Move data to the appropriate device

  model.eval() # Set the model to evaluation mode
  test_loss = 0
  with torch.no_grad():
    out_label = model(img.unsqueeze(dim = 0))
  if loss_yolo_fn:
    # Calculate the test loss if a YOLO loss function is provided
    loss_fn = loss_yolo_fn(C=1, S=7, B=3, lambda_noobj=0.5, lambda_coord=5)
    test_loss = loss_fn(out_label, labels.unsqueeze(dim=0));

  # Reshape the model output to the desired format
  out_label = out_label.reshape(7, 7, -1)
  out_label = out_label.to('cpu') # Move predictions to the CPU

  # Filter out low-confidence predictions based on probability threshold
  apples = out_labels_apple(labels = out_label, threshold = prob_threshold)

  # Perform non-maximum suppression to remove redundant predictions based on IoU thresholding
  apples = non_max_suppression(apples, threshold = iou_threshold)

  # Get image dimensions
  Ch, W, H = img.shape
  # Convert the image tensor to a PIL image for visualization
  img_pil = Image.fromarray((torch.permute(img.to('cpu'), (1, 2, 0)).numpy() * 255).astype('uint8'))
  draw = ImageDraw.Draw(img_pil)
  font = ImageFont.load_default()

  # Draw bounding boxes (ellipses) around detected objects on the image
  for class_idx, prob, x, y, r in apples:
    draw.ellipse((x*W-r*W, y*H-r*H, x*W+r*W, y*H+r*H), outline='blue', width=2)

    # TO DO:
    #color = 'red'
    #label_text = f"{prob:.2f}"
    #text_position = ((x*W-r*W*2,  y*H-r*H*2))
    #len_name = len(label_text)
    #draw.text(text_position, label_text, font=font, fill='white')

  # Return the annotated PIL image and the test loss
  return img_pil, float(test_loss)


def out_labels_apple(labels, threshold = 0.5, S=7, B=3, C=1):
  """
  Returns bounding boxes for the most confident objects (apples) from the YOLO output labels.

  Args:
      labels (torch.Tensor): YOLO output labels with shape [S, S, C + 4*B], where
          S is the grid size,
          C is the number of classes,
          and B is the number of bounding boxes per grid cell.
      threshold (float, optional): The confidence threshold for filtering out low-confidence predictions.
          Defaults to 0.5.
      S (int, optional): The grid size. Defaults to 7.
      B (int, optional): The number of bounding boxes per grid cell. Defaults to 3.
      C (int, optional): The number of classes. Defaults to 1.

  Returns:
      list: A list of bounding boxes for the most confident objects, represented as [class_idx, probability, x, y, r].
  """

  apples = []

  for I in range(S):
      for J in range(S):
        best_bbox = 0
        best_confidence = 0

        for bbox in range(B):
              if labels[I][J][C+4*bbox] > best_confidence:
                best_bbox = bbox
                best_confidence = labels[I][J][C+4*bbox]
        best_confidence = labels[I][J][C]

        if best_confidence >= threshold:
          class_idx = torch.argmax(labels[I][J][:C])
          prob_xywh = labels[I][J][C+(best_bbox*4):C+(best_bbox*4)+4] # P, X, Y, R
          apples.append(torch.cat((torch.tensor([J, I, class_idx]), prob_xywh), dim=0))
  #                                 I        J       C         P        X        Y        W        H
  #print(apples[0]) -> [tensor([ 0.0000,  0.0000, 14.0000,  1.1343,  0.7461,  0.7626,  0.8706,  0.7561])
  for i in range(len(apples)):
    apples[i] = [float(apples[i][2]),# Class,
                float(apples[i][3]),# Probability
                float((apples[i][0]+apples[i][4])/S),# X in whole Image
                float((apples[i][1]+apples[i][5])/S),# Y in whole Image
                float(apples[i][6])]# R in whole Image
  return apples